In [1]:
%pip install -qU langchain-google-genai
%pip install -qU langchain langchain_community

In [46]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

gemini = userdata.get('GEMINI_KEY')

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = gemini

llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash-exp",
    temperature = 0.1
)

In [3]:
from langchain.agents import tool

@tool
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    print("add is called")
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    print("mul is called")
    return a * b

@tool
def subtract(a: int, b: int) -> int:
    """Subtract two integers.

    Args:
        a: First integer
        b: Second integer
    """
    print("sub is called")
    return a - b

@tool
def divide(a: int, b: int) -> int:
    """Divide two integers.

    Args:
        a: First integer
        b: Second integer
    """
    print("divide is called")
    return a / b

@tool
def llm_res(a: str) -> str:
  """ Give general response from the LLM.

  Args:
    a: a simple query
    """
  return llm.invoke(a).content

tools = [add,subtract,multiply,divide]
llm_with_tools = llm.bind_tools(tools)

In [4]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [97]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory


memory = ConversationBufferMemory()
agent = create_tool_calling_agent(llm, tools, prompt)
executor = AgentExecutor(agent = agent, tools = tools, memory = memory)

In [93]:
while True:
    query = input("Calculator is on:(type 'off' to turn off) ")

    # Break the loop if the user inputs "off"
    if query.lower() == "off":
        break

    # Process the user's query
    res = executor.invoke({"input": query})
    print(res.get("output"))

print("Calculator is off.")

Calculator is on:(type 'off' to turn off) what is 10 time 7 and 100 added to it
mul is called
add is called
10 times 7 is 70, and 100 added to that is 170.
Calculator is on:(type 'off' to turn off) off
Calculator is off.


In [94]:
print(memory.load_memory_variables({}).get("history"))

Human: 90 - 65
AI: 25
Human: 30-8
AI: 22
Human: what is langchain
AI: I am sorry, I cannot provide information about Langchain. My capabilities are limited to performing calculations using the provided tools.
Human: 
AI: I can perform addition, subtraction, multiplication, and division. What would you like to calculate?
Human: 1-0
AI: 1
Human: 45/9
AI: 45 / 9 = 5
Human: what is 10 time 7 and 100 added to it
AI: 10 times 7 is 70, and 100 added to that is 170.
